In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [4]:

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [5]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [6]:
net = Net().to(device)

In [7]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [8]:

for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

[1,   100] loss: 1.090
[1,   200] loss: 0.634
[1,   300] loss: 0.551
[1,   400] loss: 0.542
[1,   500] loss: 0.480
[1,   600] loss: 0.469
[1,   700] loss: 0.441
[1,   800] loss: 0.438
[1,   900] loss: 0.436
[2,   100] loss: 0.394
[2,   200] loss: 0.388
[2,   300] loss: 0.390
[2,   400] loss: 0.374
[2,   500] loss: 0.384
[2,   600] loss: 0.369
[2,   700] loss: 0.372
[2,   800] loss: 0.355
[2,   900] loss: 0.369
[3,   100] loss: 0.319
[3,   200] loss: 0.316
[3,   300] loss: 0.340
[3,   400] loss: 0.325
[3,   500] loss: 0.321
[3,   600] loss: 0.322
[3,   700] loss: 0.307
[3,   800] loss: 0.316
[3,   900] loss: 0.311
[4,   100] loss: 0.276
[4,   200] loss: 0.287
[4,   300] loss: 0.285
[4,   400] loss: 0.295
[4,   500] loss: 0.295
[4,   600] loss: 0.286
[4,   700] loss: 0.283
[4,   800] loss: 0.292
[4,   900] loss: 0.286
[5,   100] loss: 0.258
[5,   200] loss: 0.261
[5,   300] loss: 0.264
[5,   400] loss: 0.255
[5,   500] loss: 0.254
[5,   600] loss: 0.269
[5,   700] loss: 0.261
[5,   800] 

In [10]:

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [11]:
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 90 %
